<a href="https://colab.research.google.com/github/EllouziMedAmin/GUI-Based-Face-Recognition-Attendance-System-Using-Python-and-TensorFlow/blob/main/Train_Custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# create_unknown_dataset.py
import numpy as np
import cv2
import os
from sklearn.datasets import fetch_lfw_people

# 1. Setup Folder
save_dir = "dataset/unknown"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 2. Download Data (this might take a minute the first time)
print("Downloading Labeled Faces in the Wild (LFW) dataset...")
lfw_people = fetch_lfw_people(min_faces_per_person=1, resize=None) # Get original images

# 3. Save 50 Random Faces
print(f"Dataset loaded. Total faces available: {len(lfw_people.images)}")
indices = np.random.choice(len(lfw_people.images), 50, replace=False)

count = 0
for i in indices:
    # LFW images are in 0-1 float range, convert to 0-255 for saving
    face_img = (lfw_people.images[i] * 255).astype(np.uint8)

    # Resize to match your webcam crop (224x224)
    face_img = cv2.resize(face_img, (224, 224))

    # Save
    file_path = os.path.join(save_dir, f"unknown_{count}.jpg")
    cv2.imwrite(file_path, face_img)
    count += 1

print(f"Successfully saved {count} images to {save_dir}")

Dataset loaded. Total faces available: 13233
Successfully saved 50 images to dataset/unknown


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!mkdir -p /content/dataset/Mehdi

In [7]:
!rsync -av /content/drive/MyDrive/Mehdi/ /content/dataset/Mehdi/


sending incremental file list
./
IMG-20251214-WA0001.jpg
IMG-20251214-WA0003.jpg
IMG-20251214-WA0004.jpg
IMG-20251214-WA0005.jpg
IMG-20251214-WA0006.jpg
IMG-20251214-WA0007.jpg
IMG-20251214-WA0008.jpg
IMG-20251214-WA0009.jpg
IMG-20251214-WA0010.jpg
IMG-20251214-WA0011.jpg
IMG-20251214-WA0012.jpg
IMG-20251214-WA0013.jpg
IMG-20251214-WA0014.jpg
IMG-20251214-WA0015.jpg
IMG-20251214-WA0016.jpg
IMG-20251214-WA0017.jpg
IMG-20251214-WA0018.jpg
IMG-20251214-WA0019.jpg
IMG-20251214-WA0020.jpg
IMG-20251214-WA0021.jpg
IMG-20251214-WA0022.jpg
IMG-20251214-WA0023.jpg
IMG-20251214-WA0024.jpg
IMG-20251214-WA0025.jpg
WhatsApp Image 2025-12-14 à 15.47.05_59da9b47.jpg
WhatsApp Image 2025-12-14 à 15.59.54_61ab9ab1.jpg
WhatsApp Image 2025-12-14 à 15.59.54_8134665a.jpg
WhatsApp Image 2025-12-14 à 15.59.54_f9278437.jpg
WhatsApp Image 2025-12-14 à 15.59.56_03c91f0f.jpg
WhatsApp Image 2025-12-14 à 15.59.56_2568a9c3.jpg
WhatsApp Image 2025-12-14 à 15.59.56_524fca3f.jpg
WhatsApp Image 2025-12-14 à 15.59

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

In [6]:
# 1. Setup Data Generators (Data Augmentation adds "Effort" points)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary', # Mehdi vs Unknown
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)



Found 88 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'Amin': 0, 'unknown': 1}

In [8]:
# 2. Build the Model (Transfer Learning)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False # Freeze base model


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x) # Prevents overfitting
predictions = Dense(1, activation='sigmoid')(x) # Binary output (0 or 1)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [10]:
# 3. Train
print("Starting training...")
model.fit(train_generator, epochs=5, validation_data=validation_generator)
# 4. Save
model.save('attendance_model_mehdi_only.h5')
print("Model saved as attendance_model.h5")


Starting training...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 42s 11s/step - accuracy: 0.5110 - loss: 0.8410 - val_accuracy: 1.0000 - val_loss: 0.1852
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 732ms/step - accuracy: 0.9556 - loss: 0.2181 - val_accuracy: 1.0000 - val_loss: 0.0663
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 427ms/step - accuracy: 0.9584 - loss: 0.1087 - val_accuracy: 1.0000 - val_loss: 0.0178
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 431ms/step - accuracy: 1.0000 - loss: 0.0589 - val_accuracy: 1.0000 - val_loss: 0.0280
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 431ms/step - accuracy: 1.0000 - loss: 0.0331 - val_accuracy: 1.0000 - val_loss: 0.0045


Model saved as attendance_model.h5
